### Libraries and Importing

In [170]:
# The latest versions of the following libraries should be installed
!pip install plotnine
!pip install pmdarima

In [171]:
# Mounting google drive to google collab notebook (Only necessary in google collab environment)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [182]:
# Import relevant libraries (These should be installed if not already)
import pandas as pd
from pmdarima.arima import auto_arima
from pmdarima.arima import ARIMA as arima_order
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import warnings
import pickle
warnings.filterwarnings("ignore")

# Read csv files from data preparation
state_year = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/state_year.csv')
li_price = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/li_price.csv')
miso_load = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/miso_load.csv')
model_sales = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model_sales.csv')
state_year = state_year.sort_values(['Year', 'State'])
us_ev_sales = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/US_EV_SalesData.csv')
state_year

,State,Year,Gasoline Price,Median Income,Population,Renewable Energy Use,Total Energy Use,Transportation Energy Use,Stations Opened
2652,AK,1960,NaN,NaN,NaN,6800.0,296.0,27139.0,NaN
2653,AL,1960,NaN,NaN,NaN,112809.0,15485.0,176015.0,NaN
2654,AR,1960,NaN,NaN,NaN,48104.0,5662.0,104652.0,NaN
2655,AZ,1960,NaN,NaN,NaN,36181.0,6138.0,116689.0,NaN
2656,CA,1960,NaN,NaN,NaN,270161.0,57270.0,1224448.0,NaN
...,...,...,...,...,...,...,...,...,...
3202,VA,2022,NaN,NaN,NaN,NaN,NaN,NaN,50.0
3204,VT,2022,NaN,NaN,NaN,NaN,NaN,NaN,6.0
3206,WA,2022,NaN,NaN,NaN,NaN,NaN,NaN,33.0
3208,WI,2022,NaN,NaN,NaN,NaN,NaN,NaN,14.0


### EV Sales vs Socioeconomic Factors

In [173]:
# Clean dataframes
states = ['AL', 'AR', 'IL', 'IN', 'IA', 'KY', 'LA', 'MI', 'MN', 'MS', 'MO','ND', 'SD', 'TX', 'WI']
state_year_copy = state_year.loc[state_year['State'].isin(states)]
state_year_copy.loc[state_year_copy['Stations Opened'].isna(), 'Stations Opened'] = 0
state_year_copy = state_year_copy.dropna()
state_year_copy = state_year_copy.loc[(state_year_copy['Year'] >= 2011)]
state_year_copy = state_year_copy.reset_index().drop('index', axis=1)

us_ev_sales_copy = us_ev_sales.sort_values(['Year', 'State'])
us_ev_sales_copy = us_ev_sales_copy.loc[(us_ev_sales_copy['Year'] <= 2019)]
us_ev_sales_copy = us_ev_sales_copy.reset_index().drop('index', axis=1)

extracted_col = us_ev_sales_copy["Total"]
state_year_copy = state_year_copy.join(extracted_col)
state_year_copy['States_Cat'] = pd.factorize(state_year_copy['State'])[0]
state_year_copy = state_year_copy[['States_Cat', 'Year', 'Gasoline Price', 'Median Income', 'Population', 'Renewable Energy Use', 
                                   'Transportation Energy Use', 'Stations Opened', 'Total', 'State']]
state_year_copy


,States_Cat,Year,Gasoline Price,Median Income,Population,Renewable Energy Use,Transportation Energy Use,Stations Opened,Total,State
0,0,2011,27.53,25082.0,4799642.0,255849.0,474771.0,8.0,73,AL
1,1,2011,27.86,23039.0,2941038.0,124987.0,284546.0,18.0,22,AR
2,2,2011,28.21,28663.0,3066772.0,668883.0,307194.0,18.0,49,IA
3,3,2011,28.71,32857.0,12867783.0,313512.0,985537.0,11.0,328,IL
4,4,2011,27.74,29475.0,6517250.0,222942.0,581270.0,5.0,94,IN
...,...,...,...,...,...,...,...,...,...,...
130,10,2019,19.03,16413.0,2978227.0,66398.0,342707.0,12.0,273,MS
131,11,2019,21.60,21205.0,763724.0,210222.0,131355.0,10.0,114,ND
132,12,2019,21.05,18142.0,887127.0,261949.0,98484.0,10.0,186,SD
133,13,2019,19.07,23743.0,28986794.0,962899.0,3334081.0,242.0,5780,TX


In [174]:
# Perform feature extraction to see how well each variable x predicts for Y
array = state_year_copy.values
X = array[:,0:8]
Y = array[:,8]

test = SelectKBest(score_func=f_classif, k=5)
fit = test.fit(X, Y)
# summarize scores
for i in range(len(fit.scores_)):
  print(state_year_copy.columns[i] + ": " + str(fit.scores_[i]))
features = fit.transform(X)




States_Cat: 12.38733552631579
Year: 1.6750637755102042
Gasoline Price: 0.8847555164713398
Median Income: 3.214787784399947
Population: 31.80435584690869
Renewable Energy Use: 0.8497372786675979
Transportation Energy Use: 32.36896829539239
Stations Opened: 28.584300255490326


In [175]:
us_ev_sales.groupby('Year').sum().reset_index()

,Year,BEV sales,PHEV sales,Total
0,2011,583,2101,2684
1,2012,1599,7537,9136
2,2013,4951,6619,11570
3,2014,4818,7537,12355
4,2015,5598,5146,10744
5,2016,7262,8265,15527
6,2017,9629,9291,18920
7,2018,23571,12313,35884
8,2019,21210,8287,29497
9,2020,32931,6998,39929


In [176]:
socio = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/socioeconomic_min_aicc_results.csv')
socio

,Year,State,Attribute,Model Parameters,Prediction Years Out,Prediction,Confidence Interval,AIC,AICc,BIC,Actual,Error,Absolute Error
0,2010,AL,Gasoline Price,"(0, 1, 0)",1,1.843360e+01,[14.16 22.707],113.920192,114.441932,116.357944,21.72,-3.286400,3.286400
1,2010,AL,Median Income,"(1, 1, 1)",1,2.438891e+04,[19632.499 29145.313],465.642492,467.547254,470.517996,27196.00,-2807.093683,2807.093683
2,2010,AL,Population,"(0, 2, 0)",1,4.798463e+06,[4777359.349 4819566.984],517.754640,518.276379,520.110748,4785514.00,12949.166667,12949.166667
3,2010,AL,Renewable Energy Use,"(0, 2, 2)",1,2.591495e+05,[193942.053 324357.011],580.722882,582.627644,585.435097,242243.00,16906.531742,16906.531742
4,2010,AL,Stations Opened,"(0, 1, 0)",1,-5.023221e-06,[-1.605e-05 6.003e-06],-510.596471,-510.074732,-508.158719,0.00,-0.000005,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9895,2029,WI,Median Income,"(0, 1, 0)",10,1.240400e+04,[-1227.288 26035.288],459.742146,460.263886,462.179898,NaN,NaN,NaN
9896,2029,WI,Population,"(0, 2, 0)",10,5.894939e+06,[5728804.652 6061074.015],473.917352,474.439092,476.273460,NaN,NaN,NaN
9897,2029,WI,Renewable Energy Use,"(2, 2, 1)",10,2.482845e+05,[-183105.207 679674.124],540.514051,543.514051,546.404320,NaN,NaN,NaN
9898,2029,WI,Stations Opened,"(0, 1, 2)",10,5.177209e+01,[27.156 76.388],193.170455,195.075217,198.045959,NaN,NaN,NaN


In [177]:
state_year_total_group = state_year_copy.groupby('Year').sum().reset_index()
state_year_total_group

,Year,States_Cat,Gasoline Price,Median Income,Population,Renewable Energy Use,Transportation Energy Use,Stations Opened,Total
0,2011,105,425.24,423369.0,96220158.0,3438866.0,9124562.0,271.0,2684
1,2012,105,433.05,413096.0,96885748.0,3380374.0,9007770.0,379.0,9136
2,2013,105,421.92,402764.0,97531189.0,3623547.0,9299687.0,151.0,11570
3,2014,105,402.61,397772.0,98207597.0,3781147.0,9376613.0,197.0,12355
4,2015,105,287.16,384044.0,98850012.0,3847568.0,9514314.0,397.0,10744
5,2016,105,255.67,361738.0,99443605.0,4082961.0,9718689.0,375.0,15527
6,2017,105,288.43,345210.0,99959551.0,4329303.0,9740683.0,471.0,18920
7,2018,105,319.20,333808.0,100383822.0,4521642.0,9887781.0,633.0,35884
8,2019,105,301.75,315582.0,100822979.0,4619094.0,9994140.0,770.0,29497


In [178]:
i=0
train_exogenous = state_year_total_group.loc[(state_year_total_group['Year'] < 2016+i) &\
                  (state_year_total_group['Year'] >= 2011+i), ].drop(['Total', 'States_Cat'], axis=1)
train_exogenous

,Year,Gasoline Price,Median Income,Population,Renewable Energy Use,Transportation Energy Use,Stations Opened
0,2011,425.24,423369.0,96220158.0,3438866.0,9124562.0,271.0
1,2012,433.05,413096.0,96885748.0,3380374.0,9007770.0,379.0
2,2013,421.92,402764.0,97531189.0,3623547.0,9299687.0,151.0
3,2014,402.61,397772.0,98207597.0,3781147.0,9376613.0,197.0
4,2015,287.16,384044.0,98850012.0,3847568.0,9514314.0,397.0


In [179]:
socio['Train Year'] = socio['Year'] - socio['Prediction Years Out']
socio[['State', 'Year', 'Attribute', \
                                      'Prediction',  \
                                      'Prediction Years Out']].pivot\
                                      (index=["State", "Year", \
                                      "Prediction Years Out"], \
                                       columns='Attribute', values="Prediction")\
                                       .reset_index().loc[(socio['Train Year'] == 2016+i) &\
          (socio['Prediction Years Out'] <= 4)]

Attribute,State,Year,Prediction Years Out,Gasoline Price,Median Income,Population,Renewable Energy Use,Stations Opened,Transportation Energy Use


In [180]:
exogenous_features = ["Gasoline Price", "Population", "Median Income", "Renewable Energy Use", "Transportation Energy Use", "Stations Opened"]

test_exogenous = socio[['State', 'Year', 'Attribute', \
                                      'Prediction',  \
                                      'Prediction Years Out', 'Train Year']].pivot\
                                      (index=["State", "Year", \
                                      "Prediction Years Out", "Train Year"], \
                                       columns='Attribute', values="Prediction")\
                                       .reset_index()
test_exogenous = test_exogenous.loc[(test_exogenous['Train Year'] == 2016+i) & (test_exogenous['Prediction Years Out'] == 1)][exogenous_features]
summer = test_exogenous.sum()
summer.name = 'sum'
test_exogenous = test_exogenous.append(summer)
test_exogenous.tail(1)

Attribute,Gasoline Price,Population,Median Income,Renewable Energy Use,Transportation Energy Use,Stations Opened
sum,260.5188,1.000197e+08,342216.842116,4.263956e+06,9.854421e+06,417.186487


In [183]:
# Setup dataframe columns
df_dict = dict()
df_dict['Year'] = []
df_dict["Attribute"] = []
df_dict["Model Parameters"] = []
df_dict["Prediction Years Out"] = []
df_dict["Prediction"] = []
df_dict["Confidence Interval"] = []
df_dict["AIC"] = []
df_dict["AICc"] = []
df_dict["BIC"] = []
exogenous_features = ["Gasoline Price", "Population", "Median Income", "Renewable Energy Use", "Transportation Energy Use", "Stations Opened"]
state_year_total_group = state_year_copy.groupby('Year').sum().reset_index()
state_year_total_group
arimax_national_models = []

# Setup p, d, q hyperparameters
p_vals = [0, 1, 2]
# Differencing was found to be unecessary
d_vals = [0]
q_vals = [0, 1, 2]

# Repeat below loops 4 times to predict 4 years into the future for each of
# 2016 to 2020
for i in range(4):
  print("Predicting for", str(2016 + i))
  # Predict value for each relevant attribute
  for col in ['Total']:
    # Predict value for each of MISOs states
      train = state_year_total_group.loc[(state_year_total_group['Year'] < 2016+i) &\
                                          (state_year_total_group['Year'] >= 2011+i), "Total"]

      train_exogenous = state_year_total_group.loc[(state_year_total_group['Year'] < 2016+i) &\
                  (state_year_total_group['Year'] >= 2011+i), ].drop(['Total', 'States_Cat'], axis=1)

      test_exogenous = socio[['State', 'Year', 'Attribute', \
                                      'Prediction',  \
                                      'Prediction Years Out', 'Train Year']].pivot\
                                      (index=["State", "Year", \
                                      "Prediction Years Out", "Train Year"], \
                                       columns='Attribute', values="Prediction")\
                                       .reset_index()
 
      # Collect exogenous attributes from socio economic predictions
      test_exog = pd.DataFrame()
      row = None
      for ii in range(4):
        test_exogenous2 = test_exogenous.loc[(test_exogenous['Train Year'] == 2016+i) & \
                                                  (test_exogenous['Prediction Years Out'] == ii+1)][exogenous_features]
        summer = test_exogenous2.sum()
        summer.name = 'sum'
        test_exogenous2 = test_exogenous2.append(summer)
        row = test_exogenous2.tail(1)

        test_exog = test_exog.append(row)

      # Predict for each combination of p, d, q
      for p in p_vals:
        for d in d_vals:
          for q in q_vals:
            try:
              model = arima_order(order=(p, d, q))
              print("Train", train)
              print("Train exogenous", train_exogenous[exogenous_features])
              print("Test exogenous", test_exog[exogenous_features])
              
              model.fit(train, exogenous=train_exogenous[exogenous_features])
              arimax_national_models.append([2016+i, col, p, d, q, pickle.dumps(model)])
              prediction, confint = model.predict(n_periods=4, return_conf_int=True, exogenous=test_exog[exogenous_features])
              print("Prediction", prediction)

            except Exception as e:
              # If model crashes (Linear algebra error or not normalized)
              # Store results
              for ii in range(4):
                df_dict["Year"].append(2016 + i + ii)
                df_dict["Attribute"].append(col)
                df_dict["Model Parameters"].append((p, d, q))
                df_dict["Prediction Years Out"].append(ii+1)
                df_dict["Prediction"].append(None)
                df_dict["Confidence Interval"].append(None)
                df_dict["AIC"].append(None)
                df_dict["AICc"].append(None)
                df_dict["BIC"].append(None)
              continue

            # Store results
            for ii in range(len(prediction)):
              df_dict["Year"].append(2016 + i + ii)
              df_dict["Attribute"].append(col)
              df_dict["Model Parameters"].append((p, d, q))
              df_dict["Prediction Years Out"].append(ii+1)
              df_dict["Prediction"].append(prediction[ii])
              df_dict["Confidence Interval"].append(confint[ii])
              try:
                aic = False
                aicc = False
                bic = False
                aic = model.aic()
                aicc = model.aicc()
                bic = model.bic()
                df_dict["AIC"].append(aic)
                df_dict["AICc"].append(aicc)
                df_dict["BIC"].append(bic)
              except Exception:
                if aic:
                  df_dict["AIC"].append(aic)
                else:
                  df_dict["AIC"].append(None)
                if aicc:
                  df_dict["AICc"].append(aicc)
                else:
                  df_dict["AICc"].append(None)
                if bic: 
                  df_dict["BIC"].append(bic)
                else:
                  df_dict["BIC"].append(None)
                continue
# Convert dictionary to pandas dataframe
df3 = pd.DataFrame(df_dict)
df3

Predicting for 2016
Train 0     2684
1     9136
2    11570
3    12355
4    10744
Name: Total, dtype: int64
Train exogenous    Gasoline Price  Population  Median Income  Renewable Energy Use  \
0          425.24  96220158.0       423369.0             3438866.0   
1          433.05  96885748.0       413096.0             3380374.0   
2          421.92  97531189.0       402764.0             3623547.0   
3          402.61  98207597.0       397772.0             3781147.0   
4          287.16  98850012.0       384044.0             3847568.0   

   Transportation Energy Use  Stations Opened  
0                  9124562.0            271.0  
1                  9007770.0            379.0  
2                  9299687.0            151.0  
3                  9376613.0            197.0  
4                  9514314.0            397.0  
Test exogenous Attribute  Gasoline Price    Population  Median Income  Renewable Energy Use  \
sum              260.5188  1.000197e+08  342216.842116          4.263956e

,Year,Attribute,Model Parameters,Prediction Years Out,Prediction,Confidence Interval,AIC,AICc,BIC
0,2016,Total,"(0, 0, 0)",1,18105.308511,"[18105.30849104737, 18105.30853024665]",-89.939725,-125.939725,-93.064222
1,2017,Total,"(0, 0, 0)",2,22720.077296,"[22720.07727593153, 22720.07731513081]",-89.939725,-125.939725,-93.064222
2,2018,Total,"(0, 0, 0)",3,28172.983241,"[28172.983221316463, 28172.983260515743]",-89.939725,-125.939725,-93.064222
3,2019,Total,"(0, 0, 0)",4,33141.801924,"[33141.80190458761, 33141.80194378689]",-89.939725,-125.939725,-93.064222
4,2016,Total,"(0, 0, 1)",1,18105.308511,"[18105.30849105262, 18105.3085302519]",-87.939724,-123.939724,-91.454782
...,...,...,...,...,...,...,...,...,...
139,2022,Total,"(2, 0, 1)",4,29274.458378,"[29274.458355532635, 29274.458400966756]",-83.528346,-121.242631,-87.824529
140,2019,Total,"(2, 0, 2)",1,31982.047709,"[31982.04768936457, 31982.047728563848]",-81.939845,-120.939845,-86.626590
141,2020,Total,"(2, 0, 2)",2,31313.475130,"[31313.475110186308, 31313.475149385587]",-81.939845,-120.939845,-86.626590
142,2021,Total,"(2, 0, 2)",3,28852.906836,"[28852.90681668969, 28852.90685588897]",-81.939845,-120.939845,-86.626590


In [185]:
textfile = open("/content/us_ev_sales_models_national.text", "w")
for element in arimax_national_models:
  textfile.write(str(element) + "\n")
textfile.close()

In [156]:
# Store actual values for EV Sales
tot_us_ev_sales = us_ev_sales.groupby('Year').sum().reset_index()

df3["Actual"] = "NA"
for year in range(2016, 2022):
  df3.loc[(df3["Year"] == year) & \
          (df3["Attribute"] == "Total"), \
          "Actual"] = tot_us_ev_sales.loc[\
          (tot_us_ev_sales['Year'] == year), "Total"].values[0]


# Store prediction error
df3['Error'] = None
for year in range(2016, 2022):
  for attribute in list(set(df3['Attribute'])):
    df3.loc[(df3["Year"] == year) & \
            (df3["Attribute"] == attribute), \
            "Error"] = df3.loc[(df3["Year"] == year) & \
            (df3["Attribute"] == attribute), \
            "Prediction"] - df3.loc[(df3["Year"] == year) & \
            (df3["Attribute"] == attribute), \
            "Actual"]

# Store absolute prediction error
df3['Absolute Error'] = None
for year in range(2016, 2022):
  for attribute in list(set(df3['Attribute'])):
    df3.loc[(df3["Year"] == year) & \
            (df3["Attribute"] == attribute), \
            "Absolute Error"] = abs(df3.loc[(df3["Year"] == year) & \
            (df3["Attribute"] == attribute), \
            "Error"])
df3.to_csv('arimax_results_total.csv', index=False)
df3

,Year,Attribute,Model Parameters,Prediction Years Out,Prediction,Confidence Interval,AIC,AICc,BIC,Actual,Error,Absolute Error
0,2016,Total,"(0, 0, 0)",1,18105.308511,"[18105.30849104737, 18105.30853024665]",-89.939725,-125.939725,-93.064222,15527,2578.308511,2578.308511
1,2017,Total,"(0, 0, 0)",2,22720.077296,"[22720.07727593153, 22720.07731513081]",-89.939725,-125.939725,-93.064222,18920,3800.077296,3800.077296
2,2018,Total,"(0, 0, 0)",3,28172.983241,"[28172.983221316463, 28172.983260515743]",-89.939725,-125.939725,-93.064222,35884,-7711.016759,7711.016759
3,2019,Total,"(0, 0, 0)",4,33141.801924,"[33141.80190458761, 33141.80194378689]",-89.939725,-125.939725,-93.064222,29497,3644.801924,3644.801924
4,2016,Total,"(0, 0, 1)",1,18105.308511,"[18105.30849105262, 18105.3085302519]",-87.939724,-123.939724,-91.454782,15527,2578.308511,2578.308511
...,...,...,...,...,...,...,...,...,...,...,...,...
139,2022,Total,"(2, 0, 1)",4,29274.458378,"[29274.458355532635, 29274.458400966756]",-83.528346,-121.242631,-87.824529,NA,None,None
140,2019,Total,"(2, 0, 2)",1,31982.047709,"[31982.04768936457, 31982.047728563848]",-81.939845,-120.939845,-86.626590,29497,2485.047709,2485.047709
141,2020,Total,"(2, 0, 2)",2,31313.475130,"[31313.475110186308, 31313.475149385587]",-81.939845,-120.939845,-86.626590,39929,-8615.52487,8615.52487
142,2021,Total,"(2, 0, 2)",3,28852.906836,"[28852.90681668969, 28852.90685588897]",-81.939845,-120.939845,-86.626590,93921,-65068.093164,65068.093164


In [167]:
df3_best = df3.loc[df3.loc[df3['Prediction Years Out'] < 6].groupby(['Year', 'Attribute', 'Prediction Years Out']).AICc.idxmin()]
df3_best.to_csv('best_arimax_results_total.csv', index=False)
df3_best_errors = df3_best.loc[df3_best['Year'] < 2022]
df3_best_errors['Absolute Percentage Error'] = (abs(df3_best_errors['Actual'].astype('float32') - df3_best_errors['Prediction'].astype('float32')) / df3_best_errors['Actual'].astype('float32')) * 100

In [169]:
df3_best_mean = df3_best_errors.astype({'Error': 'float32', 'Absolute Error': 'float32', 'Absolute Percentage Error': 'float32'}).loc[df3_best['Year'] < 2022].groupby('Prediction Years Out').mean()
df3_best_mean[['Error', 'Absolute Error', 'Absolute Percentage Error']].to_csv('mean_arimax_results.csv', index=True)
df3_best_mean

,Year,Prediction,AIC,AICc,BIC,Error,Absolute Error,Absolute Percentage Error
Prediction Years Out,,,,,,,,
1,2017.5,25085.846542,-89.939617,-125.939617,-93.064114,128.846619,7019.049805,28.057487
2,2018.5,28538.213427,-89.939617,-125.939617,-93.064114,-2519.286377,4419.325195,14.798497
3,2019.5,31211.518000,-89.939617,-125.939617,-93.064114,-18596.232422,24862.568359,42.166344
4,2020.0,36923.116056,-89.939540,-125.939540,-93.064037,-17525.882812,26118.308594,35.070255
